In [12]:
import scanpy as sc
import pandas as pd
import numpy as np
from pathlib import Path
import numpy.testing as npt

In [2]:
scvi_adatas = list(Path("../results/scvi/").glob("*.h5ad"))

In [3]:
def make_df():
    for path in scvi_adatas:
        filename = path.stem
        dataset, rest = filename.split(".")
        _, node, cpus, _ = rest.split("_", maxsplit=3)
        yield dataset, node, cpus, str(path)

samplesheet = pd.DataFrame.from_records(make_df(), columns=["dataset", "node", "cpus", "adata_path"])

In [10]:
adatas = {
    r.Index: sc.read_h5ad(r.adata_path) for r in samplesheet.itertuples()
}

In [14]:
samplesheet

,dataset,node,cpus,adata_path
0,Guo_Zhang_2018_NSCLC,apollo-new,40cpus,../results/scvi/Guo_Zhang_2018_NSCLC.qc_apollo...
1,Lambrechts_2018_LUAD_6653,apollo-new,40cpus,../results/scvi/Lambrechts_2018_LUAD_6653.qc_a...
2,Guo_Zhang_2018_NSCLC,apollo-old,40cpus,../results/scvi/Guo_Zhang_2018_NSCLC.qc_apollo...
3,Guo_Zhang_2018_NSCLC,apollo-old,20cpus,../results/scvi/Guo_Zhang_2018_NSCLC.qc_apollo...
4,Guo_Zhang_2018_NSCLC,zeus,20cpus,../results/scvi/Guo_Zhang_2018_NSCLC.qc_zeus_2...
5,Maynard_Bivona_2020_NSCLC,apollo-new,40cpus,../results/scvi/Maynard_Bivona_2020_NSCLC.qc_a...
6,Lambrechts_2018_LUAD_6653,apollo-old,20cpus,../results/scvi/Lambrechts_2018_LUAD_6653.qc_a...
7,Lambrechts_2018_LUAD_6653,apollo-old,40cpus,../results/scvi/Lambrechts_2018_LUAD_6653.qc_a...
8,Maynard_Bivona_2020_NSCLC,apollo-old,40cpus,../results/scvi/Maynard_Bivona_2020_NSCLC.qc_a...
9,Guo_Zhang_2018_NSCLC,apollo-new,20cpus,../results/scvi/Guo_Zhang_2018_NSCLC.qc_apollo...


Things are reproducible on the same CPU generation, with the same number of cores, but that's about it

In [18]:
npt.assert_almost_equal(adatas[9].obsm["X_scVI"], adatas[4].obsm["X_scVI"])

Same CPU, different number of cores:

In [19]:
npt.assert_almost_equal(adatas[2].obsm["X_scVI"], adatas[3].obsm["X_scVI"])

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 114250 / 114250 (100%)
Max absolute difference: 4.551491
Max relative difference: 9721.013
 x: array([[-0.4055374, -1.243769 ,  0.5031881, ...,  0.1723216,  0.3767376,
        -0.3098114],
       [-2.1020555,  0.6172941,  0.4834996, ...,  2.0647378, -0.0233086,...
 y: array([[-0.5382577, -1.4259595,  0.5230535, ...,  0.2197821,  0.0816827,
        -0.4511819],
       [-2.0066655,  0.9846088,  0.2316375, ...,  2.0555646, -0.0033504,...

Same number of cores, different CPUs

In [20]:
npt.assert_almost_equal(adatas[3].obsm["X_scVI"], adatas[4].obsm["X_scVI"])

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 114250 / 114250 (100%)
Max absolute difference: 3.8100638
Max relative difference: 22365.818
 x: array([[-0.5382577, -1.4259595,  0.5230535, ...,  0.2197821,  0.0816827,
        -0.4511819],
       [-2.0066655,  0.9846088,  0.2316375, ...,  2.0555646, -0.0033504,...
 y: array([[-0.4074667, -1.3004682,  0.4541413, ...,  0.2611647,  0.1430842,
        -0.1908449],
       [-2.001299 ,  0.4988387,  0.4894946, ...,  2.142696 ,  0.3412218,...

And while the values are correlated, they